In [1]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import pandas as pd
from scipy.special import expit
from matplotlib import pyplot as plt
import numpy as np
import bw_processing as bwp
from carculator import *

import pandas as pd


In [2]:
def replacement(cf_mass):
    return 5 - cf_mass * (1 + expit(cf_mass * 4 - 5)) + np.random.rand()

In [3]:
class AluminiumThing:
    def __next__(self):
        cf = np.random.rand() * 2.5
        alu = replacement(cf)
        return np.array([cf, alu])

# Ecoinvent importer 

In [4]:
if 'biosphere-3' in bd.databases:
    del bd.databases['biosphere-3']

In [5]:
bd.projects.set_current("project")

In [6]:
ei = bd.database.DatabaseChooser('ecoinvent 3.7 cutoff')

bi.bw2setup()

ei = bi.SingleOutputEcospold2Importer(r"C:/Users/lorenzou/Box Sync/PhD work/Brightway/ei/datasets/",
                                           'ecoinvent 3.7 cutoff'
                                          )
ei.apply_strategies()
ei.statistics()

ei.all_linked

ei.write_database()

# Read data from carculator

In [22]:
cip = CarInputParameters()
cip.static()

dcts, array = fill_xarray_from_input_parameters(cip)
array = array.interp(year = [2020,2040], kwargs = {'fill_value':'extrapolate'})
cm = CarModel(array, cycle = 'WLTC')
cm.set_all()     

#back_config = {
 #   'country':'CN',
 #   'custom electricity mix': [[.2,0,.1,.025,.025,.05,.4,.2,0,0,0,0,0,0,0],
                              # [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]#[.35,.1,.05,.1,.1,0,.2,.1,0,0,0,0,0,0,0]]
#}


#array = array.interp(year = [2020,2040], kwargs = {'fill_value':'extrapolate'})
 
scope = {
    'powertrain':['ICEV-d'],
    'size:':['Large']
}

ic = InventoryCalculation(cm.array, scope = scope)
i, _ = ic.export_lci_to_bw(ecoinvent_version = '3.7')


#ic_ICEV.export_lci_to_excel(filename = 'ICEVs_inventories',
#                            software_compatibility='brightway2',
#                            ecoinvent_version='3.7')


****************** IMPORTANT BACKGROUND PARAMETERS ******************
 * The country of use is RER
 * Power and energy batteries produced in CN using NMC-111 chemistry.
 * in 2020, % of renewable: 0%, GHG intensity per kWh: 0 g. CO2-eq.
 	 * in 2040, % of renewable: 0%, GHG intensity per kWh: 0 g. CO2-eq.
 * diesel is completed by biodiesel - cooking oil.
 	 * in 2020 _________________________________________ 8.0%
 	 * in 2040 _________________________________________ 20.0%
 * *********************************************************************


0% [##############################] 100% | ETA: 00:00:00 | Item ID: 967
Total time elapsed: 00:00:00


In [23]:
my_interface = AluminiumThing()
next(my_interface)

array([0.99626747, 4.02762197])

if "mobility" in bd.databases:
    del bd.databases['mobility']
    
if "biosphere" in bd.databases:
    del bd.databases['biosphere']    

In [24]:
if 'new_biosphere' in bd.databases:
    del bd.databases['new_biosphere']
if 'carculator db' in bd.databases:
    del bd.databases['carculator db']
    i.apply_strategies()
    
i.match_database('ecoinvent 3.7 cutoff', fields = ('name','unit','location','reference product'))
i.match_database('biosphere-3', fields  = ('name','unit','categories'))
i.match_database(fields = ('name','unit','location','reference product'))
i.match_database(fields  = ('name','unit','categories'))
i.create_new_biosphere('new_biosphere', relink = True)
i.statistics()
i.write_database()


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: assign_only_product_as_production
Applying strategy: strip_biosphere_exc_locations
Applied 4 strategies in 0.00 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Creating new biosphere database: new_biosphere


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 10/20/2021 17:45:21
  Finished: 10/20/2021 17:45:21
  Total time elapsed: 00:00:00
  CPU %: 98.90
  Memory %: 2.93
Applying strategy: link_iterable_by_fields
Applied 1 strategies in 0.05 seconds
287 datasets
4561 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 10/20/2021 17:45:29
  Finished: 10/20/2021 17:45:30
  Total time elapsed: 00:00:00
  CPU %: 92.00
  Memory %: 2.94
Created database: carculator db


Brightway2 SQLiteBackend: carculator db

i_bev.match_database('ecoinvent 3.7 cutoff', fields = ('name','unit','location','reference product'))

i_bev.match_database('biosphere-3', fields  = ('name','unit','categories'))
i_bev.match_database(fields = ('name','unit','location','reference product'))
i_bev.match_database(fields  = ('name','unit','categories'))
i_bev.create_new_biosphere('biosphere_bev', relink = True)
i_bev.statistics()
i_bev.write_database()


db = bd.Database("mobility")
db.register()

biosphere = bd.Database("biosphere-3")
biosphere.register()

In [25]:
biosphere = bd.Database('new_biosphere')
biosphere.register()

In [26]:
db = bd.Database('carculator db')
db.register()

In [38]:
{o.id: o for o in db if 'fuel' in  str(o)}

{24672: 'Ethanol, from wheat grains, at fuelling station' (kilogram, RER, None),
 24821: 'gasoline production, synthetic, from methanol, hydrogen from electrolysis, CO2 from DAC, economic allocation, at fuelling station' (kilogram, RER, None),
 24721: 'Hydrogen, gaseous, 700 bar, from SMR of biogas, at fuelling station' (kilogram, RER, None),
 24725: 'Hydrogen, gaseous, 700 bar, from gasification of woody biomass in entrained flow gasifier, with CCS, at fuelling station' (kilogram, RER, None),
 24822: 'gasoline production, synthetic, from methanol, hydrogen from electrolysis, CO2 from DAC, energy allocation, at fuelling station' (kilogram, RER, None),
 24722: 'Hydrogen, gaseous, 700 bar, from coal gasification, at fuelling station' (kilogram, RER, None),
 24858: 'electricity market for fuel preparation, 2040' (kilowatt hour, RER, None),
 24855: 'electricity market for fuel preparation, 2020' (kilowatt hour, RER, None),
 24719: 'Hydrogen, gaseous, 700 bar, from SMR of NG, with CCS, at f

In [28]:
FU = [{act:1} for act in bd.Database('carculator db') if 'Passenger' in act['name'] and 'Medium' in act['name']]
FU


[{'Passenger car, diesel, Medium, 2020, EURO-6d' (unit, RER, None): 1},
 {'Passenger car, diesel, Medium, 2040, EURO-6d' (unit, RER, None): 1}]

In [34]:
lca = bc.LCA(FU[1],('IPCC 2013','climate change','GWP 100a'))
lca.lci()
lca.lcia()
pd.DataFrame([
    {'score': lca.score,}]) 


,score
0,18339.462939


In [35]:
{node['type'] for node in db}

{'process'}

In [36]:
lca.technosphere_matrix

<19415x19415 sparse matrix of type '<class 'numpy.float64'>'
	with 236960 stored elements in Compressed Sparse Row format>

In [32]:
lca.biosphere_matrix

<2389x19415 sparse matrix of type '<class 'numpy.float64'>'
	with 388659 stored elements in Compressed Sparse Row format>

In [33]:
# Build data package 
# 

In [ ]:
indices_array = np.array([(FU[0].id, FU[].id), (FU[].id, FU[0].id)], dtype=bwp.INDICES_DTYPE)
indices_array

In [ ]:
import bw2calc as bc

In [ ]:
from bw_processing import load_datapackage
from fs.zipfs import ZipFS

In [ ]:
database_dp = load_datapackage(ZipFS(db.filepath_processed()))
method_dp = load_datapackage(ZipFS(ipcc.filepath_processed()))

We can use the datapackages directly, just as before:

In [ ]:
interface_dp = bwp.create_datapackage()

In [ ]:
indices_array = np.array([(cf.id, bike.id), (alu.id, bike.id)], dtype=bwp.INDICES_DTYPE)
indices_array

In [ ]:
{o.id: o for o in db}

In [ ]:
interface_dp.add_dynamic_vector?

In [ ]:
interface_dp.add_dynamic_vector(
    matrix="technosphere_matrix",
    interface=my_interface,
    name="alu replacing CF interface",
    flip_array=np.array([True, True]),
    indices_array=indices_array,
)    

In [ ]:
lca = bc.LCA(
    demand={bike.id: 1},
    data_objs=[database_dp, method_dp, interface_dp],
    use_distributions=False,
    # use_arrays=True,
    seed_override=None
)
lca.lci()
lca.lcia()

In [ ]:
pd.DataFrame([
    {
        'score': lca.score, 
        'inv': lca.inventory.sum(), 
        'char': lca.characterization_matrix.sum()
    } for _, _ in zip(lca, range(10))
])

In [ ]:
lca.technosphere_matrix.toarray()

In [ ]:
next(lca)